# Práctica Independiente: El clasificador Bagging en Scikit Learn

En scikit-learn, los métodos de bagging se ofrecen como un meta-estimador unificado de `BaggingClassifier` (respectivamente ` BaggingRegressor`), tomando como entrada un estimador de base definido por el usuario junto con parámetros que especifican la estrategia para construir subsets aleatorios.    
En particular, `max_samples` y ` max_features` controlan el tamaño de los subsets (en términos de muestras y características), mientras que `bootstrap` y ` bootstrap_features` controlan si las muestras y características se toman con o sin reemplazo.   
Cuando se utiliza un subset de las muestras disponibles, el error de generalización se puede estimar con las muestras fuera de bolsa (out-of-bag) poniendo `oob_score=True`.

Como ejemplo, vamos a comparar el rendimiento de un clasificador KNN simple versus el clasificador de Bagging en el conjunto de datos de aceptabilidad de autos.

El primer paso es leer los datos en Pandas.

In [ ]:
import pandas as pd
df = pd.read_csv('car.csv')
df.head()

In [ ]:
df.shape

### Preprocesamiento

Cada una de las catgorías parece ser un nivel de la variable. Exploren la composición de cada serie con el método value_counts() ¿Cómo parecen haber sido generadas las categorías?

#### Escalar las variables

A continuación transformemos las categorías ordinales en niveles escalares para las variables buying, maint, lug_boot y safety.

In [ ]:
df.buying = df.buying.map({'low':1,'med':2,'high':3,'vhigh':4})

In [ ]:
df.maint = df.maint.map({'low':1,'med':2,'high':3,'vhigh':4})

In [ ]:
df.lug_boot = df.lug_boot.map({'small':1,'med':2,'big':3})

In [ ]:
df.safety = df.safety.map({'low':1,'med':2,'high':3})

#### Calcular dummies

Para las variables doors y persons calculamos dummis porque hay niveles que no se pueden recodificar.

In [ ]:
df_dum = pd.get_dummies(df[['doors','persons']],drop_first =True)

In [ ]:
X = pd.concat([df_dum,df[['buying','maint','lug_boot','safety']]], axis=1)

### Clasificación binaria

Para simplificar, transformemos el problema en uno de clasificación binaria recodificando "unnacceptable" como 0 y todos los demás estados como 1. 

In [ ]:
y = df.acceptability.map({'unacc':0,'acc':1,'good':1,'vgood':1})

### Prueba de ensambles

El siguiente paso es calcular el cross_val_score en los dos clasificadores:

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import StratifiedKFold


my_cv = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

knn = KNeighborsClassifier()
bagging = BaggingClassifier(knn, random_state=1)

print ("KNN Score:\t", cross_val_score(knn, X, y, cv=my_cv, n_jobs=-1).mean())
print ("Bagging Score:\t", cross_val_score(bagging, X, y, cv=my_cv, n_jobs=-1).mean())